# Define model constants

In [ ]:
_HOLE_BOUND = ((-10, 10), (-10, 10))  # Hole boundary -10 <= x <= 10, -10 <= y <= 10

GRID_SCALE = 1
GRID_RESOLUTION = 2
GRID_W = 75
GRID_BOUND = ((-GRID_W / 2, GRID_W / 2), (-GRID_W / 2, GRID_W / 2))

DESIRED_VARIANCE = 0.975

TEST_BOUNDARY = (0, 74)

# A. Define GPR model

In [ ]:
from src.prediction.gpr_model_trainer import GPRModelTrainer

from sklearn.gaussian_process.kernels import RBF, ConstantKernel
import numpy as np

gpr_kernel = ConstantKernel(1.0, (1e-2, 1e2)) * RBF(1.0, (1e-2, 1e2))

gpr_model_trainer = GPRModelTrainer(
    gpr_kernel=gpr_kernel,
    gpr_drag_config=(10, 1e-5),
    gpr_max_temp_config=(10, 1e-3),
    gpr_avg_temp_config=(10, 1e-3),
    grid_bound=GRID_BOUND,
    grid_bound_width=GRID_W,
    grid_resolution=GRID_RESOLUTION,
    grid_scale=GRID_SCALE,
    desired_variance=DESIRED_VARIANCE,
)

# B. Define RandomForest model

In [ ]:
from src.prediction.rf_model_trainer import RfModelTrainer

rf_model_trainer = RfModelTrainer(
    rf_drag_config=(100, 42),
    rf_max_temp_config=(100, 42),
    rf_avg_temp_config=(100, 42),
    grid_bound=GRID_BOUND,
    grid_bound_width=GRID_W,
    grid_resolution=GRID_RESOLUTION,
    grid_scale=GRID_SCALE,
    desired_variance=DESIRED_VARIANCE,
)

# C. Define ETR model

In [ ]:
from src.prediction.etr_model_trainer import EtrModelTrainer

etr_model_trainer = EtrModelTrainer(
    etr_drag_config=(100, 42),
    etr_max_temp_config=(100, 42),
    etr_avg_temp_config=(100, 42),
    grid_bound=GRID_BOUND,
    grid_bound_width=GRID_W,
    grid_resolution=GRID_RESOLUTION,
    grid_scale=GRID_SCALE,
    desired_variance=DESIRED_VARIANCE,
)

# D. Define ENSEM model

In [ ]:
from src.prediction.ensem_model_trainer import EnsemModelTrainer

ensem_model_trainer = EnsemModelTrainer(
    ensem_drag_config=(42, 10, 42),
    ensem_max_temp_config=(42, 10, 42),
    ensem_avg_temp_config=(42, 10, 42),
    grid_bound=GRID_BOUND,
    grid_bound_width=GRID_W,
    grid_resolution=GRID_RESOLUTION,
    grid_scale=GRID_SCALE,
    desired_variance=DESIRED_VARIANCE,
)

# E. Define ADA model

In [ ]:
from src.prediction.ada_model_trainer import AdaModelTrainer

ada_model_trainer = AdaModelTrainer(
    ada_drag_config=(100, 42),
    ada_max_temp_config=(100, 42),
    ada_avg_temp_config=(100, 42),
    grid_bound=GRID_BOUND,
    grid_bound_width=GRID_W,
    grid_resolution=GRID_RESOLUTION,
    grid_scale=GRID_SCALE,
    desired_variance=DESIRED_VARIANCE,
)

# F. Define CRT model (Unstable)

In [ ]:
from src.prediction.crt_model_trainer import CrtModelTrainer

crt_model_trainer = CrtModelTrainer(
    crt_drag_config=42,
    crt_avg_temp_config=42,
    crt_max_temp_config=42,
    grid_bound=GRID_BOUND,
    grid_bound_width=GRID_W,
    grid_resolution=GRID_RESOLUTION,
    grid_scale=GRID_SCALE,
    desired_variance=DESIRED_VARIANCE,
)

# Collect test models

In [ ]:
from typing import Literal, Union
from src.prediction.model_trainer import ModelTrainer


test_models: list[tuple[ModelTrainer, Union[Literal["with_std", "without_std"]]]] = [
    (gpr_model_trainer, "with_std"),
    (rf_model_trainer, "without_std"),
    (etr_model_trainer, "without_std"),
    (ensem_model_trainer, "without_std"),
    (ada_model_trainer, "without_std"),
    (crt_model_trainer, "without_std"),
]

# Test suite functions

1. with `std`: gaussian based regressions
    - `test_model_with_std__train_set`: test model with respect to original train input source.
    - `test_model_with_std__pattern`: test model with respect to new `list[Pattern]` data set. 
2. without `std`: basic regressions
    - `test_model_without_std__train_set`: test model with respect to original train input source.
    - `test_model_without_std__pattern`: test model with respect to new `list[Pattern]` data set. 

3. Benchmark all models

In [ ]:
from src.geometry.pattern_unit import Pattern
from src.prediction.model_trainer import ModelTrainer
from src.prediction.to_model_input import to_model_input
from src.prediction.model_metrics import ModelMetrics


def test_model_with_std__train_set(model_trainer: ModelTrainer):
    model_trainer.train_model()
    res = model_trainer.get_train_set()
    model = model_trainer.get_model()

    if res is None:
        return

    x, y = res

    test = x
    test_res = y

    metrics = ModelMetrics(test_kit=test_res)

    drag, std1 = model[0].predict(test, return_std=True)
    met1 = metrics.drag_metrics(drag_pred=drag)  # metric : r^2

    avg_temp, std2 = model[1].predict(test, return_std=True)
    met2 = metrics.ave_temp_metrics(ave_temp_pred=avg_temp)  # metric : MSE

    max_temp, std3 = model[2].predict(test, return_std=True)
    met3 = metrics.max_temp_metrics(max_temp_pred=max_temp)  # metric : MAE

    print("-" * 100)
    print(f">> {model_trainer.model_name} test for TRAIN_SET(self) input\n")
    print(">> met: ", met1, met2, met3)

    print("-" * 100)


def test_model_without_std__train_set(model_trainer: ModelTrainer):
    model_trainer.train_model()
    model = model_trainer.get_model()
    res = model_trainer.get_train_set()

    if res is None:
        return

    x, y = res

    test = x
    test_res = y

    metrics = ModelMetrics(test_kit=test_res)

    drag = model[0].predict(test)
    met1 = metrics.drag_metrics(drag_pred=drag)  # metric : r^2

    avg_temp = model[1].predict(test)
    met2 = metrics.ave_temp_metrics(ave_temp_pred=avg_temp)  # metric : MSE

    max_temp = model[2].predict(test)
    met3 = metrics.max_temp_metrics(max_temp_pred=max_temp)  # metric : MAE

    print("-" * 100)

    print(f">> {model_trainer.model_name} test for TRAIN_SET(self) input\n")
    # print(">> real: ", test_res)
    # print(">> predicted: ", drag, avg_temp, max_temp)
    print(">> met: ", met1, met2, met3)

    print("-" * 100)


def test_model_without_std__pattern(
    model_trainer: ModelTrainer, test_pattern: list[Pattern]
):
    print("-" * 100)

    print(f"Loading {model_trainer.model_name} model, pca...")
    model = model_trainer.get_model()
    pca = model_trainer.get_pca()

    print(f">> {model_trainer.model_name} test for NEW PATTERN input\n")

    for i, pattern in enumerate(test_pattern):
        ins = to_model_input(
            pca,
            pattern_matrix=pattern.pattern_matrix,
            bound=GRID_BOUND,
            resolution=1 / GRID_RESOLUTION,
        )

        drag = model[0].predict(ins)
        avg_temp = model[1].predict(ins)
        max_temp = model[2].predict(ins)

        print(f">> test-{i} input prediction")
        print(f">> prediction: {drag}, {avg_temp}, {max_temp}")

    print("-" * 100)


def test_model_with_std__pattern(
    model_trainer: ModelTrainer, test_pattern: list[Pattern]
):
    print("-" * 100)
    print(f"Loading {model_trainer.model_name} model, pca...")
    model = model_trainer.get_model()
    pca = model_trainer.get_pca()

    print(f">> {model_trainer.model_name} test for NEW PATTERN input\n")

    for i, pattern in enumerate(test_pattern):
        ins = to_model_input(
            pca,
            pattern_matrix=pattern.pattern_matrix,
            bound=GRID_BOUND,
            resolution=1 / GRID_RESOLUTION,
        )

        drag, std = model[0].predict(ins, return_std=True)
        avg_temp, std2 = model[1].predict(ins, return_std=True)
        max_temp, std3 = model[2].predict(ins, return_std=True)

        print(f">> test-{i} input prediction")
        print(f">> prediction: {drag}, std: {std}")
        print(f">> prediction: {avg_temp}, std: {std2}")
        print(f">> prediction: {max_temp}, std: {std3}")

    print("-" * 100)


def test_model_performance_without_std(
    model_trainer: ModelTrainer, test_boundary: tuple[int, int] = TEST_BOUNDARY
):
    model_trainer.train_model(test_boundary)
    model = model_trainer.get_model()
    pca = model_trainer.get_pca()
    _, test_x_mat, test_y_mat = model_trainer.get_test_set(
        test_boundary,
        origin_pca=pca,
    )

    metrics = ModelMetrics(test_kit=test_y_mat)

    drag = model[0].predict(test_x_mat)
    met1 = metrics.drag_metrics(drag_pred=drag)  # metric : r^2

    avg_temp = model[1].predict(test_x_mat)
    met2 = metrics.ave_temp_metrics(ave_temp_pred=avg_temp)  # metric : MSE

    max_temp = model[2].predict(test_x_mat)
    met3 = metrics.max_temp_metrics(max_temp_pred=max_temp)  # metric : MAE

    print(f">> {model_trainer.model_name} test for TEST SET(perf)\n")
    print(">> met: ", met1, met2, met3)

    print("-" * 100)


def test_model_performance_with_std(
    model_trainer: ModelTrainer, test_boundary: tuple[int, int] = TEST_BOUNDARY
):
    model_trainer.train_model(test_boundary)
    model = model_trainer.get_model()
    pca = model_trainer.get_pca()
    _, test_x_mat, test_y_mat = model_trainer.get_test_set(
        test_boundary,
        origin_pca=pca,
    )

    metrics = ModelMetrics(test_kit=test_y_mat)

    drag, std1 = model[0].predict(test_x_mat, return_std=True)
    met1 = metrics.drag_metrics(drag_pred=drag)  # metric : r^2

    avg_temp, std2 = model[1].predict(test_x_mat, return_std=True)
    met2 = metrics.ave_temp_metrics(ave_temp_pred=avg_temp)  # metric : MSE

    max_temp, std3 = model[2].predict(test_x_mat, return_std=True)
    met3 = metrics.max_temp_metrics(max_temp_pred=max_temp)  # metric : MAE

    print("-" * 100)
    print(f">> {model_trainer.model_name} test for TEST SET(perf) input\n")
    print(">> met: ", met1, met2, met3)
    print(">> std: ", std1, std2, std3)
    print("-" * 100)


def bench_all_models(
    models: list[tuple[ModelTrainer, Union[Literal["with_std", "without_std"]]]],
    test_pattern: list[Pattern],
):
    for model, std in models:
        print("-" * 150)
        print("-" * 150)
        print(f"\n\n<<< Testing {model.model_name} model... >>>")
        print(f"<<< model_config: {model.train_config} >>>\n\n")

        if std == "with_std":
            test_model_performance_with_std(model)
            print("test_model_with_std__train_set finished...")
            print("\n\n")
            test_model_with_std__train_set(model)
            print("test_model_with_std__pattern finished...")
            print("\n\n")
            test_model_with_std__pattern(model, test_pattern)
            print("test_model_with_std__pattern finished...")
            print("\n\n")
        else:
            test_model_performance_without_std(model)
            print("test_model_without_std__train_set finished...")
            print("\n\n")
            test_model_without_std__train_set(model)
            print("test_model_without_std__pattern finished...")
            print("\n\n")
            test_model_without_std__pattern(model, test_pattern)
            print("test_model_without_std__pattern finished...")
            print("\n\n")
        print("-" * 150)
        print("-" * 150)
        print("\n\n")

# Generate test input data

In [ ]:
from src.grid.visualize_points import visualize_points
from src.geometry.pattern_unit import (
    PatternUnit,
    Shape,
    PatternTransformation,
    PatternTransformationMatrix,
    Pattern,
)

pattern1 = Pattern(
    pattern_transformation_matrix=PatternTransformationMatrix(
        pattern_unit=PatternUnit(
            Shape(
                w=10,
                h=7.5,
                area_functions=[lambda x, y: x**2 / 16 + y**2 / 9 <= 1],
            ),
            k=1 / GRID_RESOLUTION,
        ),
        pattern_transformation=PatternTransformation(
            "circular",
            dx=2.5,
            di=25,
            phi=np.pi / 7,
        ),
        pattern_bound=GRID_BOUND,
    ),
)

visualize_points(
    pattern1.pattern_matrix,
    k=1 / GRID_RESOLUTION,
    fig_size=(5, 5),
)

pattern2 = Pattern(
    pattern_transformation_matrix=PatternTransformationMatrix(
        pattern_unit=PatternUnit(
            Shape(
                w=10,
                h=5,
                area_functions=[lambda x, y: x**2 / 9 + y**2 / 2 <= 1],
            ),
            k=1 / GRID_RESOLUTION,
        ),
        pattern_transformation=PatternTransformation(
            "grid",
            dx=10,
            di=10 / 2,
        ),
        pattern_bound=GRID_BOUND,
    ),
)
visualize_points(
    pattern2.pattern_matrix,
    k=1 / GRID_RESOLUTION,
    fig_size=(5, 5),
)

# Run tests

In [ ]:
bench_all_models(test_models, [pattern1, pattern2])

## GPR prev result tests

### A1. `(-30, 30), res: 3`
```bash
(1, 106)
(1, 106)
>> ins1 prediction
>> prediction: [0.29497703], std: [0.00218427]
>> prediction: [392.91214535], std: [0.95447247]
>> prediction: [518.54214407], std: [2.37395765]
>> ins2 prediction
>> prediction: [0.29466794], std: [0.00233069]
>> prediction: [390.18615569], std: [1.01531236]
>> prediction: [504.76525881], std: [2.51555504]
```

### A2. `(-30, 30), res: 2`
```bash
(1, 107)
(1, 107)
>> ins1 prediction
>> prediction: [0.29583199], std: [0.00382364]
>> prediction: [383.75400754], std: [0.84201941]
>> prediction: [498.071324], std: [2.23880663]
>> ins2 prediction
>> prediction: [0.29814156], std: [0.00481413]
>> prediction: [390.5626691], std: [1.08056385]
>> prediction: [516.80923951], std: [2.82949011]
```

### B1. `(-50, 50), res: 2`
```bash
(1, 107)
(1, 107)
>> ins1 prediction
>> prediction: [0.29855318], std: [0.00413844]
>> prediction: [395.1396864], std: [0.9193178]
>> prediction: [531.81106611], std: [2.43499191]
>> ins2 prediction
>> prediction: [0.29737008], std: [0.00495669]
>> prediction: [393.6109499], std: [1.11294301]
>> prediction: [525.46977552], std: [2.90263788]
```